This is a page for listing the peat health indicators for a given site

In [ ]:
import panel as pn
import param

import utils.cards
import utils.location
import utils.site_indicators
import utils.template
import utils.utils

pn.extension()

In [ ]:
def get_card_for_phi(phi: utils.site_indicators.SiteLevelPHI) -> pn.Card:
    """Get a card for the given site level peat health indicator (PHI)."""

    data_url = utils.utils.get_url(
        "site-indicator",
        {"site-id": phi.site_id, "indicator-id": phi.name},
    )

    return pn.Card(
        objects=[
            pn.pane.HoloViews(
                phi.map(),
                width=600,
                height=500,
            ),
            pn.pane.Str(phi.description),
            pn.pane.Markdown(f"🔗 [Explore the data]({data_url})"),
        ],
        collapsed=True,
        title=phi.name,
        sizing_mode="stretch_width",
    )

In [ ]:
class Page(utils.location.UrlQueryParams):
    @param.depends("site", watch=False)
    def get_phi_cards(self) -> list:
        if self.site is None:
            return ["No indicators found"]

        directories = utils.site_indicators.all_site_level_peat_health_indicators()
        extent_names = directories.get(self.site.id, [])
        phi_objects = [utils.site_indicators.get_phi(self.site_id, extent_name) for extent_name in extent_names]
        phi_objects = [phi for phi in phi_objects if phi is not None]
        return [get_card_for_phi(phi) for phi in phi_objects]

    @param.depends("site", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Peat health indicators

                    On this page you can discover the peat health indicators for the site {site_name}.
                    
                    Check out one of the peat extent classifications below
                    to explore the peat health indicators for that extent.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Peat extent"))
            elements.extend(self.get_phi_cards())  # type: ignore

        return pn.Column(
            *elements,
            align=("start", "center"),
            min_width=400,
            max_width=1200,
            sizing_mode="stretch_width",
        )


In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main)
template.servable();